In [1]:
import requests
from scrapy.http import TextResponse
import json

In [74]:
# 쿠팡 특정상품 크롤링
search_url = "http://www.coupang.com/np/search?component=&q=샘키즈"
rep = requests.get(search_url)
response = TextResponse(rep.url, body=rep.text, encoding='utf-8')

In [75]:
# 상품마다의 링크를 불러오기에 앞서 아이템 Top 10 리스트 불러오기
products = json.loads(response.xpath('//*[@id="productList"]/@data-products').extract()[0])['indexes'][:24]

review_counts = {}
review_urls = {}
for product in products:
    # Top 10 상품마다의 상품평 개수(나중에 불러오는 정보라 상품링크 안에서는 xpath로 바로 가져올 수 없음)
    try:
        review_counts[product] = int(response.xpath('//*[@id="{}"]/a/dl/dd/div/div[4]/div[2]/span[2]/text()'.format(product)).extract()[0].strip("()"))
        review_count = int(response.xpath('//*[@id="{}"]/a/dl/dd/div/div[4]/div[2]/span[2]/text()'.format(product)).extract()[0].strip("()"))
    except Exception as e:
        print('No reviews')
#         products.remove(product)
    
    # Top 10 상품마다의 크롤링할 url(리뷰 크롤링은 '상품평 보기' 클릭 후 나타나는 reviews Request URL에서 해야함)
    if review_count > 0:
        review_urls[product] = make_url(product, review_count)
        review_url = make_url(product, review_count) # 리스트로 반환됨
    else:
        print('No reviews')   

No reviews
No reviews
No reviews


In [76]:
len(review_counts)

21

In [126]:
# products = ['32104199', '30229717','88893806','119581355', '32037268', '32033206', '30380921', '30384997']
# for product in products:
#     # Top 10 상품마다의 상품평 개수(나중에 불러오는 정보라 상품링크 안에서는 xpath로 바로 가져올 수 없음)
#     try:
#         review_counts[product] = int(response.xpath('//*[@id="{}"]/a/dl/dd/div/div[4]/div[2]/span[2]/text()'.format(product)).extract()[0].strip("()"))
#         review_count = int(response.xpath('//*[@id="{}"]/a/dl/dd/div/div[4]/div[2]/span[2]/text()'.format(product)).extract()[0].strip("()"))
#         print('{} has reviews'.format(product))
#     except Exception as e:
#         print('{} has No reviews'.format(product))
#         products.remove(product)

32104199 has reviews
30229717 has No reviews
88893806 has No reviews
119581355 has No reviews
32037268 has reviews
32033206 has reviews
30380921 has reviews
30384997 has reviews


In [2]:
def make_url(product, review_count):
    page_count = (review_count//100)+1
    review_urls = []
    for page in range(1, page_count+1):
        review_urls.append('http://www.coupang.com/vp/product/reviews?productId={}&page={}&size=100&sortBy=ORDER_SCORE_ASC&ratings=&q=&viRoleCode=2&ratingSummary=true'.format(product, page))
        # size는 100이 최대로 설정되어 있음
    return review_urls

In [18]:
search_url = "http://www.coupang.com/np/search?component=&q=샘키즈"
rep = requests.get(search_url)
response = TextResponse(rep.url, body=rep.text, encoding='utf-8')

# 상품마다의 링크를 불러오기에 앞서 아이템 Top 10 리스트 불러오기
products = json.loads(response.xpath('//*[@id="productList"]/@data-products').extract()[0])['indexes'][:5]

df = pd.DataFrame(columns=['rating', "item_nbr", "item", "option", "date", "name", "re_title", 'review']) 

review_urls = {}

for product in products:

    # Top 10 상품마다의 상품평 개수(나중에 불러오는 정보라 상품링크 안에서는 xpath로 바로 가져올 수 없음)
    try:
#         review_counts[product] = int(response.xpath('//*[@id="{}"]/a/dl/dd/div/div[4]/div[2]/span[2]/text()'.format(product)).extract()[0].strip("()"))
        review_count = int(response.xpath('//*[@id="{}"]/a/dl/dd/div/div[4]/div[2]/span[2]/text()'.format(product)).extract()[0].strip("()"))
    except Exception as e:
        print('No reviews')    

    # Top 10 상품마다의 크롤링할 url(리뷰 크롤링은 '상품평 보기' 클릭 후 나타나는 reviews Request URL에서 해야함)
    review_urls[product] = make_url(product, review_count)
    review_url = make_url(product, review_count) # 리스트로 반환됨
#     print(review_url)
    
    for url in review_url:

        review_url = url
        review_rep = requests.get(review_url)
        review_response = TextResponse(review_rep.url, body=review_rep.text, encoding='utf-8')        
        
        # Top 10 상품마다의 상품평 개수
        review_total_cnt = int(review_response.xpath('/html/body/div[2]/@data-review-total-count').extract()[0])
        
        # Top 10 상품마다 한 page의 상품평 개수
        review_url_cnt = int(float(review_response.xpath('count(/html/body/article)').extract()[0]))
        
        if review_url_cnt < 100:          
            for i in range(1, review_url_cnt+1): # 한개의 review_url에서 100개를 훑고 그 다음 review_url로 넘어가면 됨
                item_nbr = product

                date = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[2]/text()'.format(i)).extract()[0]

                name = review_response.xpath('/html/body/article[{}]/div[1]/div[2]/span/text()'.format(i)).extract()[0][:-1]

                item_option_ls = review_response.xpath('/html/body/article[{}]/div[1]/div[4]/text()'.format(i)).extract()[0].split(",")
                if len(item_option_ls) == 1:
                    item = item_option_ls[0]

                else:    
                    item = item_option_ls[0]
                    option = item_option_ls[1]

                if review_response.xpath('/html/body/article[{}]/div[3]/@class'.format(i)).extract()[0] == 'sdp-review__article__list__headline':
                    re_title = review_response.xpath('/html/body/article[{}]/div[3]/text()'.format(i)).extract()
                    review = review_response.xpath('/html/body/article[{}]/div[4]/div/text()'.format(i)).extract()

                else:
                    re_title = ['.']
                    review = review_response.xpath('/html/body/article[{}]/div[3]/div/text()'.format(i)).extract()

                rating = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[1]/div/@data-rating'.format(i)).extract()[0]

                df.loc[len(df)] = {'rating':rating, 'item_nbr':item_nbr, 'item':item, 'option':option, 'date':date, 'name':name, 're_title':re_title, 'review':review}
                
        else:
            for i in range(1, 101):
                item_nbr = product

                date = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[2]/text()'.format(i)).extract()[0]

                name = review_response.xpath('/html/body/article[{}]/div[1]/div[2]/span/text()'.format(i)).extract()[0][:-1]

                item_option_ls = review_response.xpath('/html/body/article[{}]/div[1]/div[4]/text()'.format(i)).extract()[0].split(",")
                if len(item_option_ls) == 1:
                    item = item_option_ls[0]

                else:    
                    item = item_option_ls[0]
                    option = item_option_ls[1]

                if review_response.xpath('/html/body/article[{}]/div[3]/@class'.format(i)).extract()[0] == 'sdp-review__article__list__headline':
                    re_title = review_response.xpath('/html/body/article[{}]/div[3]/text()'.format(i)).extract()
                    review = review_response.xpath('/html/body/article[{}]/div[4]/div/text()'.format(i)).extract()

                else:
                    re_title = ['.']
                    review = review_response.xpath('/html/body/article[{}]/div[3]/div/text()'.format(i)).extract()

                rating = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[1]/div/@data-rating'.format(i)).extract()[0]

                df.loc[len(df)] = {'rating':rating, 'item_nbr':item_nbr, 'item':item, 'option':option, 'date':date, 'name':name, 're_title':re_title, 'review':review}
                

In [19]:
df

,rating,item_nbr,item,option,date,name,re_title,review
0,2,32033206,샘키즈 스칸디 에디션 수납장 1005 (2종/택1),A: 화이트(A),2018.04.16,윤*영,[.],[\n \n ...
1,5,32033206,샘키즈 스칸디 에디션 수납장 1005 (2종/택1),B: 메이플(B),2018.04.08,정*영,[\n \n 외손주에게...,[\n \n ...
2,5,32033206,샘키즈 스칸디 에디션 수납장 1005 (2종/택1),B: 메이플(B),2018.05.01,손*희,[.],[\n \n ...
3,2,32033206,샘키즈 스칸디 에디션 수납장 1005 (2종/택1),A: 화이트(A),2018.02.22,강*선,[.],[\n \n ...
4,4,32033206,샘키즈 스칸디 에디션 수납장 1005 (2종/택1),B: 메이플(B),2017.12.08,강*순,[.],[\n \n ...
5,5,32033206,샘키즈 스칸디 에디션 수납장 1005 (2종/택1),B: 메이플(B),2018.01.28,l*n,[.],[\n \n ...
6,5,32033206,샘키즈 스칸디 에디션 수납장 1005 (2종/택1),A: 화이트(A),2018.01.11,손*희,[.],[\n \n ...
7,5,32033206,샘키즈 스칸디 에디션 수납장 1005 (2종/택1),A: 화이트(A),2017.12.06,이*연,[.],[\n \n ...
8,5,32033206,샘키즈 스칸디 에디션 수납장 1005 (2종/택1),B: 메이플(B),2017.11.13,김*혜,[.],[\n \n ...
9,5,32033206,샘키즈 스칸디 에디션 수납장 1005 (2종/택1),B: 메이플(B),2018.05.25,허*민,[.],[]


In [20]:
# re_title의 의미있는 문장 발라내기(re_title은 리스트 length 1이다)
  
# re_title 컬럼에 적용
def re_title_filter(a):
    for i in range(len(a[0].split('\n'))):
        if len(a[0].split('\n')[i].strip()) != 0:
            return a[0].split('\n')[i].strip()

In [21]:
# review의 리스트의 여러 요소로 나뉘어있는 문장 중 의미있는 문장 발라내고 하나의 스트링으로 합치기

def review_filter(a):
    s = ''
    for i in range(len(a)):
        for ii in range(len(a[i].split('\n'))):
            if a[i].split('\n')[ii].strip() != 0:
                s += a[i].split('\n')[ii].strip() + ' '
    return s

In [22]:
def kkma_pos(a):
    from konlpy.tag import Kkma
    kkma = Kkma()
    return kkma.pos(a)

In [23]:
def kkma_pos_filter(a):
    """
    df['pos']를 받아 품사 필터링을 거쳐 해당 키워드 리스트 반환
    """
    ls = []
    for i in range(len(a)):
        #NNG(보통 명사), NNP(고유 명사), VA(형용사), XR(어근), MAG(일반 부사), UN(명사추정범주)
        if a[i][1]=='NNG' or a[i][1]=='NNP' or a[i][1]=='VA' or a[i][1]=='UN' or a[i][1]=='XR' or a[i][1]=='MAG':
            ls.append(a[i][0])
    return ls

In [24]:
def rating_filter(a):
    if a == '4' or a == '5':
        return 1
        # 긍정
    else:
        return 0
        # 부정
    
# def __change_weekday_to_number(df):
#     weekday_dict = {
#         "Monday" : 1,
#         "Tuesday" : 2,
#         "Wednesday" : 3,
#         "Thursday" : 4,
#         "Friday" : 5,
#         "Saturday" : 6,
#         "Sunday" : 7
#     }
#     return df["Weekday"].map(weekday_dict)

In [25]:
df_ = df.copy()
df_['re_title_filtered'] = df_['re_title'].apply(re_title_filter)
df_['review_filtered'] = df_['review'].apply(review_filter)
df_['full_review'] = df_['re_title_filtered']+df_['review_filtered']
df_['pos'] = df_['full_review'].apply(kkma_pos)
df_['pos_filtered'] = df_['pos'].apply(kkma_pos_filter)
df_['rating_filtered'] = df_['rating'].apply(rating_filter)

In [12]:
df_.columns

Index(['rating', 'item_nbr', 'item', 'option', 'date', 'name', 're_title',
       'review', 're_title_filtered', 'review_filtered', 'full_review', 'pos',
       'pos_filtered', 'rating_filtered'],
      dtype='object')

In [26]:
df_fin = df_[['rating', 'rating_filtered', 'item_nbr', 'item', 'option', 'date', 'name', 'full_review', 'pos',
       'pos_filtered']]
df_fin.tail()

,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
667,5,1,30364081,샘키즈 스칸디 에디션 수납장 1305 (2종/택1),B형:메이플(B),2018.01.23,김*라,.,"[(., SF)]",[]
668,5,1,30364081,샘키즈 스칸디 에디션 수납장 1305 (2종/택1),B형:메이플(B),2018.01.07,김*신,.,"[(., SF)]",[]
669,5,1,30364081,샘키즈 스칸디 에디션 수납장 1305 (2종/택1),B형:메이플(B),2017.12.29,하*수,.,"[(., SF)]",[]
670,5,1,30364081,샘키즈 스칸디 에디션 수납장 1305 (2종/택1),A형:화이트(A),2017.12.28,서*기,.,"[(., SF)]",[]
671,5,1,30364081,샘키즈 스칸디 에디션 수납장 1305 (2종/택1),A형:화이트(A),2017.12.12,신*슬,.,"[(., SF)]",[]


In [27]:
X = df_fin['full_review']
y = df_fin['rating_filtered']

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

model1 = Pipeline([
            ('vect', CountVectorizer()), 
            ('mb', MultinomialNB()),
        ])

In [29]:
model1.fit(X, y)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('mb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [30]:
print(classification_report(y, model1.predict(X)))

             precision    recall  f1-score   support

          0       1.00      0.43      0.61        53
          1       0.95      1.00      0.98       619

avg / total       0.96      0.96      0.95       672



In [37]:
search_url = "http://www.coupang.com/np/search?component=&q=샘키즈"
rep = requests.get(search_url)
response = TextResponse(rep.url, body=rep.text, encoding='utf-8')

# 상품마다의 링크를 불러오기에 앞서 아이템 Top 10 리스트 불러오기
products = json.loads(response.xpath('//*[@id="productList"]/@data-products').extract()[0])['indexes'][5:20]

df = pd.DataFrame(columns=['rating', "item_nbr", "item", "option", "date", "name", "re_title", 'review']) 

review_urls = {}

for product in products:

    # Top 10 상품마다의 상품평 개수(나중에 불러오는 정보라 상품링크 안에서는 xpath로 바로 가져올 수 없음)
    try:
#         review_counts[product] = int(response.xpath('//*[@id="{}"]/a/dl/dd/div/div[4]/div[2]/span[2]/text()'.format(product)).extract()[0].strip("()"))
        review_count = int(response.xpath('//*[@id="{}"]/a/dl/dd/div/div[4]/div[2]/span[2]/text()'.format(product)).extract()[0].strip("()"))
    except Exception as e:
        print('No reviews')    

    # Top 10 상품마다의 크롤링할 url(리뷰 크롤링은 '상품평 보기' 클릭 후 나타나는 reviews Request URL에서 해야함)
    review_urls[product] = make_url(product, review_count)
    review_url = make_url(product, review_count) # 리스트로 반환됨
#     print(review_url)
    
    for url in review_url:

        review_url = url
        review_rep = requests.get(review_url)
        review_response = TextResponse(review_rep.url, body=review_rep.text, encoding='utf-8')        
        
        # Top 10 상품마다의 상품평 개수
        review_total_cnt = int(review_response.xpath('/html/body/div[2]/@data-review-total-count').extract()[0])
        
        # Top 10 상품마다 한 page의 상품평 개수
        review_url_cnt = int(float(review_response.xpath('count(/html/body/article)').extract()[0]))
        
        if review_url_cnt < 100:          
            for i in range(1, review_url_cnt+1): # 한개의 review_url에서 100개를 훑고 그 다음 review_url로 넘어가면 됨
                item_nbr = product

                date = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[2]/text()'.format(i)).extract()[0]

                name = review_response.xpath('/html/body/article[{}]/div[1]/div[2]/span/text()'.format(i)).extract()[0][:-1]

                item_option_ls = review_response.xpath('/html/body/article[{}]/div[1]/div[4]/text()'.format(i)).extract()[0].split(",")
                if len(item_option_ls) == 1:
                    item = item_option_ls[0]

                else:    
                    item = item_option_ls[0]
                    option = item_option_ls[1]

                if review_response.xpath('/html/body/article[{}]/div[3]/@class'.format(i)).extract()[0] == 'sdp-review__article__list__headline':
                    re_title = review_response.xpath('/html/body/article[{}]/div[3]/text()'.format(i)).extract()
                    review = review_response.xpath('/html/body/article[{}]/div[4]/div/text()'.format(i)).extract()

                else:
                    re_title = ['.']
                    review = review_response.xpath('/html/body/article[{}]/div[3]/div/text()'.format(i)).extract()

                rating = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[1]/div/@data-rating'.format(i)).extract()[0]

                df.loc[len(df)] = {'rating':rating, 'item_nbr':item_nbr, 'item':item, 'option':option, 'date':date, 'name':name, 're_title':re_title, 'review':review}
                
        else:
            for i in range(1, 101):
                item_nbr = product

                date = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[2]/text()'.format(i)).extract()[0]

                name = review_response.xpath('/html/body/article[{}]/div[1]/div[2]/span/text()'.format(i)).extract()[0][:-1]

                item_option_ls = review_response.xpath('/html/body/article[{}]/div[1]/div[4]/text()'.format(i)).extract()[0].split(",")
                if len(item_option_ls) == 1:
                    item = item_option_ls[0]

                else:    
                    item = item_option_ls[0]
                    option = item_option_ls[1]

                if review_response.xpath('/html/body/article[{}]/div[3]/@class'.format(i)).extract()[0] == 'sdp-review__article__list__headline':
                    re_title = review_response.xpath('/html/body/article[{}]/div[3]/text()'.format(i)).extract()
                    review = review_response.xpath('/html/body/article[{}]/div[4]/div/text()'.format(i)).extract()

                else:
                    re_title = ['.']
                    review = review_response.xpath('/html/body/article[{}]/div[3]/div/text()'.format(i)).extract()

                rating = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[1]/div/@data-rating'.format(i)).extract()[0]

                df.loc[len(df)] = {'rating':rating, 'item_nbr':item_nbr, 'item':item, 'option':option, 'date':date, 'name':name, 're_title':re_title, 'review':review}
                

No reviews


In [38]:
df_1 = df.copy()
df_1['re_title_filtered'] = df_1['re_title'].apply(re_title_filter)
df_1['review_filtered'] = df_1['review'].apply(review_filter)
df_1['full_review'] = df_1['re_title_filtered']+df_1['review_filtered']
df_1['pos'] = df_1['full_review'].apply(kkma_pos)
df_1['pos_filtered'] = df_1['pos'].apply(kkma_pos_filter)
df_1['rating_filtered'] = df_1['rating'].apply(rating_filter)

In [39]:
df_fin_1 = df_1[['rating', 'rating_filtered', 'item_nbr', 'item', 'option', 'date', 'name', 'full_review', 'pos',
       'pos_filtered']]
df_fin_1.tail()

,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
184,2,0,32034439,한샘 샘키즈 수납장 1005 모던_좁은형,C형 (몸통)메이플/(박스)올화이트(C),2018.01.13,신*정,.,"[(., SF)]",[]
185,5,1,32034439,한샘 샘키즈 수납장 1005 모던_좁은형,B형 (몸통)크림화이트/(박스)하프그레이(B),2018.01.13,최*숙,.,"[(., SF)]",[]
186,5,1,32034439,한샘 샘키즈 수납장 1005 모던_좁은형,A형 (몸통)크림화이트/(박스)올화이트(A),2017.12.12,김*영,.,"[(., SF)]",[]
187,5,1,32101157,[한샘] 샘키즈 2단책장 870 (2종/택1),B형:메이플(B),2017.12.29,하*수,.,"[(., SF)]",[]
188,1,0,20880794,ing가구 애슐리 밀키 수납장 아동서랍장/수납장,올화이트,2017.10.23,강*경,새걸로 샀는데 왜 헌거 껴주나요? 이게 어떻게 된거지?!! 아이꺼 정리하려고 박스...,"[(새, MDT), (것, NNB), (으로, JKM), (사, VV), (었, E...","[왜, 어떻, 거지, 아이, 정리, 박스, 꺼, 이렇, 스크레치, 제품, 검수, 작..."


In [40]:
X = df_fin_1['full_review']
y = df_fin_1['rating_filtered']

In [41]:
print(classification_report(y, model1.predict(X)))

             precision    recall  f1-score   support

          0       1.00      0.15      0.26        20
          1       0.91      1.00      0.95       169

avg / total       0.92      0.91      0.88       189



In [16]:
df_fin.loc[18]

rating                                                             5
rating_filtered                                                    1
item_nbr                                                    30366312
item                                          한샘 샘키즈 수납장 1305 블라썸 민트
option                                     A형:(몸통)크림화이트/(박스)민트그레이(A)
date                                                      2017.12.02
name                                                            초록마을
full_review        굿굿굿 굿입니다  토요일 6시 넘어 오셔서 7시까지 설치해주고 가셨네요.  월요일에...
pos                [(굿, NNG), (굿, NNG), (굿, NNG), (굿, NNG), (이, V...
pos_filtered       [굿, 굿, 굿, 굿, 토요, 설치, 월요일, 쿠팡주문, 수요일, 한샘, 고객, 센...
Name: 18, dtype: object

------------------------------------------------

### Vectorizer

In [416]:
df_fin['pos_filtered'][18]

['굿', '굿', '굿', '굿', '토요', '설치', '월요일', '쿠팡주문', '수요일', '한샘', '고객', '센터', '전화', '오고', '배송', '날짜', '오늘', '아침', '쯤', '전화', '토요일', '배송', '괜히', '정도', '열심히', '친절', '설치', '많이', '미안', '제법', '무게', '부속품', '많', '혼자', '너무', '같', '따뜻', '차', '한잔', '사양', '오히려', '본인', '시끄럽', '아기', '사과', '다', '설치', '때', '도로', '막', '인사', '배송', '기사님', '최고', '각설', '제품', '워낙에', '육아', '집', '집', '한집', '아이템', '더', '설명', '없', '바구니', '물', '한번', '세척', '수건', '물기', '하나하나', '비닐', '포장', '먼지', '별로', '없', '아기', '옷', '라', '세척', '한번', '구매', '시', '먼지', '제법', '말', '성비', '요만', '없', '구매', '색', '너무', '맘', '차곡차곡', '옷', '정리', '제법', '이리저리', '난리', '옷가지', '정리', '너무', '좋', '아기', '여름옷', '정도', '같', '내복', '정도', '외출복', '정도', '수납', '가능', '차후', '아이', '장난감', '정리', '대', '사용가능', '같', '활용도', '좋', '잘', '사용', '같']

In [406]:
vect = CountVectorizer()
vect.fit(df_fin['pos_filtered'][18])
vect.vocabulary_
# 한글자는 자동필터링 함
# ex) 굿, 많, 쯤, 같, 막, 집, 더, 물, 없, 색, 맘, 옷, 좋, 잘
# https://stackoverflow.com/questions/20717641/countvectorizer-i-not-showing-up-in-vectorized-text

{'토요': 64, '설치': 28, '월요일': 49, '쿠팡주문': 63, '수요일': 34, '한샘': 69, '고객': 2, '센터': 31, '전화': 54, '오고': 42, '배송': 18, '날짜': 7, '오늘': 43, '아침': 39, '토요일': 65, '괜히': 3, '정도': 55, '열심히': 41, '친절': 62, '많이': 12, '미안': 16, '제법': 57, '무게': 14, '부속품': 21, '혼자': 72, '너무': 9, '따뜻': 11, '한잔': 70, '사양': 24, '오히려': 44, '본인': 20, '시끄럽': 35, '아기': 36, '사과': 23, '도로': 10, '인사': 52, '기사님': 5, '최고': 61, '각설': 1, '제품': 58, '워낙에': 48, '육아': 50, '한집': 71, '아이템': 38, '설명': 27, '바구니': 17, '한번': 68, '세척': 30, '수건': 32, '물기': 15, '하나하나': 67, '비닐': 22, '포장': 66, '먼지': 13, '별로': 19, '구매': 4, '성비': 29, '요만': 47, '차곡차곡': 59, '정리': 56, '이리저리': 51, '난리': 6, '옷가지': 45, '여름옷': 40, '내복': 8, '외출복': 46, '수납': 33, '가능': 0, '차후': 60, '아이': 37, '장난감': 53, '사용가능': 26, '활용도': 73, '사용': 25}

In [409]:
vect = CountVectorizer(min_df=0, token_pattern=r"\b\w+\b")
vect.fit(df_fin['pos_filtered'][18])
vect.vocabulary_

{'굿': 6, '토요': 85, '설치': 41, '월요일': 65, '쿠팡주문': 84, '수요일': 47, '한샘': 90, '고객': 3, '센터': 44, '전화': 71, '오고': 57, '배송': 30, '날짜': 9, '오늘': 58, '아침': 53, '쯤': 78, '토요일': 86, '괜히': 4, '정도': 72, '열심히': 56, '친절': 83, '많이': 21, '미안': 28, '제법': 74, '무게': 25, '부속품': 33, '많': 20, '혼자': 93, '너무': 11, '같': 2, '따뜻': 16, '차': 79, '한잔': 91, '사양': 36, '오히려': 59, '본인': 32, '시끄럽': 49, '아기': 50, '사과': 35, '다': 12, '때': 17, '도로': 15, '막': 19, '인사': 68, '기사님': 7, '최고': 82, '각설': 1, '제품': 75, '워낙에': 64, '육아': 66, '집': 77, '한집': 92, '아이템': 52, '더': 14, '설명': 40, '없': 54, '바구니': 29, '물': 26, '한번': 89, '세척': 43, '수건': 45, '물기': 27, '하나하나': 88, '비닐': 34, '포장': 87, '먼지': 24, '별로': 31, '옷': 60, '라': 18, '구매': 5, '시': 48, '말': 22, '성비': 42, '요만': 63, '색': 39, '맘': 23, '차곡차곡': 80, '정리': 73, '이리저리': 67, '난리': 8, '옷가지': 61, '좋': 76, '여름옷': 55, '내복': 10, '외출복': 62, '수납': 46, '가능': 0, '차후': 81, '아이': 51, '장난감': 70, '대': 13, '사용가능': 38, '활용도': 94, '잘': 69, '사용': 37}

In [417]:
display(len(df_fin['pos_filtered'][18]))
display(len(list(set(df_fin['pos_filtered'][18]))))
display(len(vect.vocabulary_.keys()))

126

95

95

##### DictVectorizer

In [20]:
from sklearn.feature_extraction import DictVectorizer

In [21]:
v = DictVectorizer(sparse=False)
D = [{'A':1, 'B':2}, {'B':3, 'C':1}]
X = v.fit_transform(D)
X
# 발생가능한 모든 key를 columns로 하고, value(해당 key의 개수 등)를 row에 나타냄
# 하나의 딕셔너리가 한 행이 됨

array([[1., 2., 0.],
       [0., 3., 1.]])

In [22]:
v.feature_names_

['A', 'B', 'C']

##### CountVectorizer

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
# 1. 문서를 토큰 리스트로 변환한다
# 2. 각 문서에서 토큰의 출현 빈도를 센다
# 3. 각 문서를 BOW 인코딩 벡터로 변환한다

In [24]:
corpus = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
    'The last document?',    
]
# 하나의 리스트가 문서 하나를 의미
# 한 리스트 안의 요소 하나가 한 문장을 의미 -> 이 단위로 벡터 생성

# df_에는 여러 단어가 하나의 리스트 안에 있고
# -> 한 단어마다 array가 생성됨
# -> index마다 array를 합쳐주어야 함

# 띄어쓰기가 잘 되어있지 않은 문장에서는 countvectorizer만으로는 단어를 잘 골라내지 못함
# 중복된 단어 걸러줄 필요 없음(max_df, min_df의 판단기준이 되어주기도 해야함)
# 단 한 문장 안에서 중복해서 나온 단어는 자동으로 1로 카운트
# 여기서는 한 단어를 한 문장으로(리스트의 요소 하나 단위) 치기 때문에 상관 없다
# max_df=1.0, min_df=1 디폴트

In [25]:
# https://stackoverflow.com/questions/27697766/understanding-min-df-and-max-df-in-scikit-countvectorizer

# max_df is used for removing terms that appear too frequently, also known as "corpus-specific stop words". For example:

# max_df = 0.50 means "ignore terms that appear in more than 50% of the documents".
# max_df = 25 means "ignore terms that appear in more than 25 documents".
# The default max_df is 1.0, which means "ignore terms that appear in more than 100% of the documents".
# Thus, the default setting does not ignore any terms.

# min_df is used for removing terms that appear too infrequently. For example:

# min_df = 0.01 means "ignore terms that appear in less than 1% of the documents".
# min_df = 5 means "ignore terms that appear in less than 5 documents".
# The default min_df is 1, which means "ignore terms that appear in less than 1 document".
# Thus, the default setting does not ignore any terms.

In [26]:
df_fin['pos_filtered'][18]

['굿',
 '굿',
 '굿',
 '굿',
 '토요',
 '설치',
 '월요일',
 '쿠팡주문',
 '수요일',
 '한샘',
 '고객',
 '센터',
 '전화',
 '오고',
 '배송',
 '날짜',
 '오늘',
 '아침',
 '쯤',
 '전화',
 '토요일',
 '배송',
 '괜히',
 '정도',
 '열심히',
 '친절',
 '설치',
 '많이',
 '미안',
 '제법',
 '무게',
 '부속품',
 '많',
 '혼자',
 '너무',
 '같',
 '따뜻',
 '차',
 '한잔',
 '사양',
 '오히려',
 '본인',
 '시끄럽',
 '아기',
 '사과',
 '다',
 '설치',
 '때',
 '도로',
 '막',
 '인사',
 '배송',
 '기사님',
 '최고',
 '각설',
 '제품',
 '워낙에',
 '육아',
 '집',
 '집',
 '한집',
 '아이템',
 '더',
 '설명',
 '없',
 '바구니',
 '물',
 '한번',
 '세척',
 '수건',
 '물기',
 '하나하나',
 '비닐',
 '포장',
 '먼지',
 '별로',
 '없',
 '아기',
 '옷',
 '라',
 '세척',
 '한번',
 '구매',
 '시',
 '먼지',
 '제법',
 '말',
 '성비',
 '요만',
 '없',
 '구매',
 '색',
 '너무',
 '맘',
 '차곡차곡',
 '옷',
 '정리',
 '제법',
 '이리저리',
 '난리',
 '옷가지',
 '정리',
 '너무',
 '좋',
 '아기',
 '여름옷',
 '정도',
 '같',
 '내복',
 '정도',
 '외출복',
 '정도',
 '수납',
 '가능',
 '차후',
 '아이',
 '장난감',
 '정리',
 '대',
 '사용가능',
 '같',
 '활용도',
 '좋',
 '잘',
 '사용',
 '같']

In [27]:
vect = CountVectorizer()
vect.fit(df_fin['pos_filtered'][18])
vect.vocabulary_
# 한글자는 자동필터링 함
# ex) 굿, 많, 쯤, 같, 막, 집, 더, 물, 없, 색, 맘, 옷, 좋, 잘
# https://stackoverflow.com/questions/20717641/countvectorizer-i-not-showing-up-in-vectorized-text

{'가능': 0,
 '각설': 1,
 '고객': 2,
 '괜히': 3,
 '구매': 4,
 '기사님': 5,
 '난리': 6,
 '날짜': 7,
 '내복': 8,
 '너무': 9,
 '도로': 10,
 '따뜻': 11,
 '많이': 12,
 '먼지': 13,
 '무게': 14,
 '물기': 15,
 '미안': 16,
 '바구니': 17,
 '배송': 18,
 '별로': 19,
 '본인': 20,
 '부속품': 21,
 '비닐': 22,
 '사과': 23,
 '사양': 24,
 '사용': 25,
 '사용가능': 26,
 '설명': 27,
 '설치': 28,
 '성비': 29,
 '세척': 30,
 '센터': 31,
 '수건': 32,
 '수납': 33,
 '수요일': 34,
 '시끄럽': 35,
 '아기': 36,
 '아이': 37,
 '아이템': 38,
 '아침': 39,
 '여름옷': 40,
 '열심히': 41,
 '오고': 42,
 '오늘': 43,
 '오히려': 44,
 '옷가지': 45,
 '외출복': 46,
 '요만': 47,
 '워낙에': 48,
 '월요일': 49,
 '육아': 50,
 '이리저리': 51,
 '인사': 52,
 '장난감': 53,
 '전화': 54,
 '정도': 55,
 '정리': 56,
 '제법': 57,
 '제품': 58,
 '차곡차곡': 59,
 '차후': 60,
 '최고': 61,
 '친절': 62,
 '쿠팡주문': 63,
 '토요': 64,
 '토요일': 65,
 '포장': 66,
 '하나하나': 67,
 '한번': 68,
 '한샘': 69,
 '한잔': 70,
 '한집': 71,
 '혼자': 72,
 '활용도': 73}

In [28]:
vect = CountVectorizer(token_pattern=r"\b\w+\b")
vect.fit(df_fin['pos_filtered'][18])
vect.vocabulary_

{'가능': 0,
 '각설': 1,
 '같': 2,
 '고객': 3,
 '괜히': 4,
 '구매': 5,
 '굿': 6,
 '기사님': 7,
 '난리': 8,
 '날짜': 9,
 '내복': 10,
 '너무': 11,
 '다': 12,
 '대': 13,
 '더': 14,
 '도로': 15,
 '따뜻': 16,
 '때': 17,
 '라': 18,
 '막': 19,
 '많': 20,
 '많이': 21,
 '말': 22,
 '맘': 23,
 '먼지': 24,
 '무게': 25,
 '물': 26,
 '물기': 27,
 '미안': 28,
 '바구니': 29,
 '배송': 30,
 '별로': 31,
 '본인': 32,
 '부속품': 33,
 '비닐': 34,
 '사과': 35,
 '사양': 36,
 '사용': 37,
 '사용가능': 38,
 '색': 39,
 '설명': 40,
 '설치': 41,
 '성비': 42,
 '세척': 43,
 '센터': 44,
 '수건': 45,
 '수납': 46,
 '수요일': 47,
 '시': 48,
 '시끄럽': 49,
 '아기': 50,
 '아이': 51,
 '아이템': 52,
 '아침': 53,
 '없': 54,
 '여름옷': 55,
 '열심히': 56,
 '오고': 57,
 '오늘': 58,
 '오히려': 59,
 '옷': 60,
 '옷가지': 61,
 '외출복': 62,
 '요만': 63,
 '워낙에': 64,
 '월요일': 65,
 '육아': 66,
 '이리저리': 67,
 '인사': 68,
 '잘': 69,
 '장난감': 70,
 '전화': 71,
 '정도': 72,
 '정리': 73,
 '제법': 74,
 '제품': 75,
 '좋': 76,
 '집': 77,
 '쯤': 78,
 '차': 79,
 '차곡차곡': 80,
 '차후': 81,
 '최고': 82,
 '친절': 83,
 '쿠팡주문': 84,
 '토요': 85,
 '토요일': 86,
 '포장': 87,
 '하나하나': 88,
 '한번': 89,
 '한샘': 90,
 '한잔'

In [29]:
display(len(df_fin['pos_filtered'][18]))
display(len(list(set(df_fin['pos_filtered'][18]))))
display(len(vect.vocabulary_.keys()))

126

95

95

In [66]:
corpus_ls = []
for i in range(len(df_fin['pos_filtered'])):
    for ii in range(len(df_fin['pos_filtered'][i])):
        corpus_ls.append(df_fin['pos_filtered'][i][ii])

In [31]:
display(corpus_ls[:20])
display(len(corpus_ls))

['아기방',
 '물건',
 '정리',
 '구매',
 '물건',
 '지정',
 '배송',
 '일',
 '근데',
 '가구',
 '찍힘',
 '설치',
 '사',
 '잘',
 '그닥',
 '견고',
 '같',
 '기분',
 '썩',
 '좋']

6719

In [36]:
vect = CountVectorizer(token_pattern=r"\b\w+\b")
vect.fit(corpus_ls)
display(len(vect.vocabulary_.keys()))
display(vect.vocabulary_)

1387

{'아기방': 758,
 '물건': 453,
 '정리': 1070,
 '구매': 120,
 '지정': 1141,
 '배송': 493,
 '일': 985,
 '근데': 156,
 '가구': 7,
 '찍힘': 1167,
 '설치': 647,
 '사': 581,
 '잘': 1024,
 '그닥': 143,
 '견고': 75,
 '같': 42,
 '기분': 173,
 '썩': 749,
 '좋': 1107,
 '교환': 116,
 '생각': 633,
 '외손주': 903,
 '선물': 641,
 '베이': 504,
 '비시': 567,
 '터': 1254,
 '집': 1154,
 '먼저': 413,
 '팬티': 1282,
 '손수건': 674,
 '물': 452,
 '티슈': 1275,
 '상부': 616,
 '장난감': 1032,
 '하부': 1316,
 '무게': 435,
 '꽤나': 204,
 '어른': 815,
 '인': 980,
 '제가': 1078,
 '밀기': 467,
 '용도': 918,
 '곳': 100,
 '저렴': 1045,
 '주시': 1123,
 '기사님': 176,
 '친절': 1218,
 '늦': 269,
 '불친절': 556,
 '방향': 490,
 '다시': 279,
 '진짜': 1150,
 '써비스': 748,
 '맘': 402,
 '안': 778,
 '박스': 475,
 '기': 166,
 '고': 87,
 '그냥': 142,
 '처음': 1183,
 '그렇': 147,
 '조금': 1090,
 '뻑뻑': 579,
 '괜찮': 109,
 '많이': 398,
 '국': 131,
 '튼튼': 1269,
 '아이': 770,
 '옷': 888,
 '수납': 686,
 '아주': 774,
 '용이': 919,
 '굿': 137,
 '토요': 1255,
 '월요일': 927,
 '쿠팡주문': 1240,
 '수요일': 692,
 '한샘': 1327,
 '고객': 88,
 '센터': 659,
 '전화': 1060,
 '오고': 872,
 '날짜': 

In [33]:
vect = CountVectorizer(token_pattern=r"\b\w+\b", min_df=2)
vect.fit(corpus_ls)
display(len(vect.vocabulary_.keys()))
display(vect.vocabulary_)

632

{'ㅇ': 0,
 'ㅈ': 1,
 'ㅋ': 2,
 'ㅎ': 3,
 'ㅠ': 4,
 '가격': 5,
 '가구': 6,
 '가능': 7,
 '가볍': 8,
 '가심': 9,
 '가요': 10,
 '가장': 11,
 '가지도': 12,
 '감': 13,
 '감도': 14,
 '감사': 15,
 '감안': 16,
 '갑': 17,
 '강력': 18,
 '강추': 19,
 '같': 20,
 '같이': 21,
 '개': 22,
 '개별': 23,
 '거실': 24,
 '거의': 25,
 '걱정': 26,
 '건': 27,
 '걸': 28,
 '검색': 29,
 '검수': 30,
 '겉': 31,
 '견고': 32,
 '결정': 33,
 '겸': 34,
 '겸사': 35,
 '경우': 36,
 '계속': 37,
 '고': 38,
 '고객': 39,
 '고맙': 40,
 '고민': 41,
 '고생': 42,
 '고정': 43,
 '곧': 44,
 '곳': 45,
 '공간': 46,
 '공장': 47,
 '과': 48,
 '과정': 49,
 '괜찮': 50,
 '굉장히': 51,
 '교구': 52,
 '교환': 53,
 '구리': 54,
 '구매': 55,
 '구매자': 56,
 '구멍': 57,
 '구입': 58,
 '구지': 59,
 '국': 60,
 '국민': 61,
 '국민장': 62,
 '굿': 63,
 '규': 64,
 '그': 65,
 '그날': 66,
 '그냥': 67,
 '그닥': 68,
 '그럼': 69,
 '그렇': 70,
 '그레': 71,
 '그레이': 72,
 '그리': 73,
 '그리고': 74,
 '근데': 75,
 '글': 76,
 '금': 77,
 '금방': 78,
 '금새': 79,
 '급하': 80,
 '기': 81,
 '기간': 82,
 '기대': 83,
 '기분': 84,
 '기사': 85,
 '기사님': 86,
 '기사분': 87,
 '기서': 88,
 '기억': 89,
 '기저귀': 90,
 '기존': 91,
 '길': 92,
 '깊

In [37]:
corpus_set = list(set(corpus_ls))
display(len(corpus_set))

# 중복된 단어들이 모두 있는 corpus_ls로 fit해서 만들어진 vocabulary는
# 중복단어 없이 만들어진 단어 집합이고
# 이를 df_['pos_filtered']에 적용시킨 후 .toarray().sum(axis=0)하면
# 한 row마다 중복된 단어의 개수까지 카운팅된 하나의 벡터가 만들어진다

1387

In [38]:
display(vect.transform(df_fin['pos_filtered'][0]).toarray())
display(vect.transform(df_fin['pos_filtered'][0]).toarray().shape)
display(vect.transform(df_fin['pos_filtered'][0]).toarray().sum(axis=0))
display(vect.transform(df_fin['pos_filtered'][0]).toarray().sum(axis=0).shape)
le = len(vect.vocabulary_.keys())
display(vect.transform(df_fin['pos_filtered'][0]).toarray().sum(axis=0).reshape(1, le))
display(vect.transform(df_fin['pos_filtered'][0]).toarray().sum(axis=0).reshape(1, le).shape)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

(22, 1387)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

(1387,)

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

(1, 1387)

In [39]:
vect.vocabulary_.keys() # 잘나오는 단어 몇개만 걸러야겠다

dict_keys(['아기방', '물건', '정리', '구매', '지정', '배송', '일', '근데', '가구', '찍힘', '설치', '사', '잘', '그닥', '견고', '같', '기분', '썩', '좋', '교환', '생각', '외손주', '선물', '베이', '비시', '터', '집', '먼저', '팬티', '손수건', '물', '티슈', '상부', '장난감', '하부', '무게', '꽤나', '어른', '인', '제가', '밀기', '용도', '곳', '저렴', '주시', '기사님', '친절', '늦', '불친절', '방향', '다시', '진짜', '써비스', '맘', '안', '박스', '기', '고', '그냥', '처음', '그렇', '조금', '뻑뻑', '괜찮', '많이', '국', '튼튼', '아이', '옷', '수납', '아주', '용이', '굿', '토요', '월요일', '쿠팡주문', '수요일', '한샘', '고객', '센터', '전화', '오고', '날짜', '오늘', '아침', '쯤', '토요일', '괜히', '정도', '열심히', '미안', '제법', '부속품', '많', '혼자', '너무', '따뜻', '차', '한잔', '사양', '오히려', '본인', '시끄럽', '아기', '사과', '다', '때', '도로', '막', '인사', '최고', '각설', '제품', '워낙에', '육아', '한집', '아이템', '더', '설명', '없', '바구니', '한번', '세척', '수건', '물기', '하나하나', '비닐', '포장', '먼지', '별로', '라', '시', '말', '성비', '요만', '색', '차곡차곡', '이리저리', '난리', '옷가지', '여름옷', '내복', '외출복', '가능', '차후', '대', '사용가능', '활용도', '사용', '최저가', '너무너무', '감사', '출산', '예정일', '여기저기', '검색', '쿠팡', '방문', '시간', '잘못', '외출', '시간대', '길래', '부랴부랴

In [74]:
def Vectorizer(df, pos_filtered_column):
    """
    1) apply 함수아님 (df와 column을 input받아야 하고, 결과가 한 column에 담기는 것이 아니기 때문)
    2) df_fin의 'pos_filtered' column을 받아 vectorized df를 생성해주는 함수
    """
    df_ = df.copy()
    corpus_ls = []
    for i in range(len(df_[pos_filtered_column])):
        for ii in range(len(df_[pos_filtered_column][i])):
            corpus_ls.append(df_[pos_filtered_column][i][ii])
            
    from sklearn.feature_extraction.text import CountVectorizer
    vect = CountVectorizer(token_pattern=r"\b\w+\b") # 한 글자도 corpus에 포함될 수 있게 해주는 정규표현식
    vect.fit(corpus_ls)

    vect_ls = []
    for i in range(len(df_)):
        vect_ls.append((vect.transform(df_[pos_filtered_column][i]).toarray().sum(axis=0).tolist()))
    
    df_vec = pd.DataFrame(vect_ls)
    return df_vec

In [75]:
X = Vectorizer(df_fin, 'pos_filtered')

In [77]:
y = df_fin['rating_filtered']

In [84]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

model = MultinomialNB()
model.fit(X, y)
y_pred = model.predict(X)
print(classification_report(y, y_pred))

             precision    recall  f1-score   support

          0       0.95      0.38      0.54        53
          1       0.95      1.00      0.97       619

avg / total       0.95      0.95      0.94       672



In [71]:
df_vec

,0,1,2,3,4,5,6,7,8,9,...,1211,1212,1213,1214,1215,1216,1217,1218,1219,1220
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
vect.transform(df_['pos_filtered'][0]).toarray().sum(axis=0).tolist()
# .reshape(1, le)[0]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [44]:
df_['pos_filtered'][0]

['아기방',
 '물건',
 '정리',
 '구매',
 '물건',
 '지정',
 '배송',
 '일',
 '근데',
 '가구',
 '찍힘',
 '설치',
 '사',
 '잘',
 '그닥',
 '견고',
 '같',
 '기분',
 '썩',
 '좋',
 '교환',
 '생각']

In [ ]:
# >>> import pandas as pd
# >>> list=[['a','b']]
# >>> list.append(['e','f'])
# >>> df=pd.DataFrame(list,columns=['col1','col2'])
#    col1 col2
# 0    a    b
# 1    e    f

In [43]:
Vectorizer(df_fin, 'pos_filtered')

TypeError: 'method' object is not subscriptable

In [ ]:
df_['rating_filtered'] = df_['rating'].apply(rating_filter)

In [ ]:
# 위 코드로 feature matrix만들어서 X, y만들면 됨

------------------------------------------------